In [1]:
import os
import warnings

import folium
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
from shapely.geometry import Point

from network_wrangler import RoadwayNetwork
from network_wrangler import ProjectCard

from ipywidgets import HBox, VBox, Output
from IPython.display import display, HTML
%config IPCompleter.greedy=True
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
#if you need to reset variables in the notebook
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [22]:
import logging
import sys
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

In [14]:
STPAUL_DIR = os.path.join(os.path.dirname(os.getcwd()),'examples','stpaul')
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR,"shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR,"link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR,"node.geojson")

In [15]:
net = RoadwayNetwork.read(link_file= STPAUL_LINK_FILE, node_file=STPAUL_NODE_FILE, shape_file=STPAUL_SHAPE_FILE, fast=True)

In [16]:
easy_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        ],
     'A':{'osm_node_id': '187899923'}, # start searching for segments at A
     'B':{'osm_node_id': '187865924'}, # end at B
}

longer_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        ],
     'A':{'osm_node_id': '187899923'}, # start searching for segments at A
     'B':{'osm_node_id': '187942339'}, # end at B
}
##TODO
multi_criteria_selection = \
{
    'link':[
        {'name': ['6th', 'Sixth', 'sixth']}, #find streets that have one of the various forms of 6th
        {'lanes': [1,2]}
         ],
     'A':{'osm_node_id': '187899923'}, # start searching for segments at A
     'B':{'osm_node_id': '187942339'}, # end at B
}

In [18]:
# test out to make sure selection query is working correctly
ProjectCard.build_link_selection_query(multi_criteria_selection, unique_model_link_identifiers=RoadwayNetwork.UNIQUE_MODEL_LINK_IDENTIFIERS, ignore=['name'])

'((lanes==1 or lanes==2) and drive_access==1)'

In [77]:
# default is 10, but we may want to set it higher here for debugging purposes
# in reality, we probably want to set it lower
RoadwayNetwork.MAX_SEARCH_BREADTH = 15

net.select_roadway_features(easy_selection)
net.select_roadway_features(longer_selection)
net.select_roadway_features(multi_criteria_selection)

easy_query = ProjectCard.build_link_selection_query(easy_selection,unique_model_link_identifiers=RoadwayNetwork.UNIQUE_MODEL_LINK_IDENTIFIERS)
A_id, B_id = net.orig_dest_nodes_foreign_key(easy_selection)
easy_key = (easy_query, A_id, B_id)

longer_query = ProjectCard.build_link_selection_query(longer_selection,unique_model_link_identifiers=RoadwayNetwork.UNIQUE_MODEL_LINK_IDENTIFIERS)
A_id, B_id = net.orig_dest_nodes_foreign_key(longer_selection)
longer_key = (longer_query, A_id, B_id)

multi_query = ProjectCard.build_link_selection_query(multi_criteria_selection,unique_model_link_identifiers=RoadwayNetwork.UNIQUE_MODEL_LINK_IDENTIFIERS)
A_id, B_id = net.orig_dest_nodes_foreign_key(multi_criteria_selection)
multi_key = (multi_query, A_id, B_id)                                                   

2019-12-04 10:06:02, DEBUG: validating selection
2019-12-04 10:06:02, DEBUG: Selection Key: ('((name.str.contains("6th") or name.str.contains("Sixth") or name.str.contains("sixth")) and drive_access==1)', 62146, 45691)
2019-12-04 10:06:02, DEBUG: validating selection
2019-12-04 10:06:02, DEBUG: Selection Key: ('((name.str.contains("6th") or name.str.contains("Sixth") or name.str.contains("sixth")) and drive_access==1)', 62146, 77073)
2019-12-04 10:06:02, DEBUG: validating selection
2019-12-04 10:06:02, DEBUG: Selection Key: ('((name.str.contains("6th") or name.str.contains("Sixth") or name.str.contains("sixth")) and (lanes==1 or lanes==2) and drive_access==1)', 62146, 77073)


In [79]:
# Here for exploring the graph

G = net.selections[easy_key]['graph']
#G.edges
#nx.get_edge_attributes(G,'weight')
#ox.plot_graph(G, fig_height=10, fig_width=16)

In [80]:
class selection_map:
    
    def __init__(self, net, selection={}):
        self.net = net
        self.selection = selection
        self.query = ProjectCard.build_link_selection_query(selection,unique_model_link_identifiers=RoadwayNetwork.UNIQUE_MODEL_LINK_IDENTIFIERS)
        
        self.selection_key = net.build_selection_key(self.selection)
        
        self.G     = self.net.selections[self.selection_key]['graph']
        self.links = self.net.selections[self.selection_key]['candidate_links']
        self.A_name = self.selection_key[1]
        self.B_name = self.selection_key[2]
        
        self.A     = self.G.nodes[self.A_name]
        self.B     = self.G.nodes[self.B_name]
        
        self.sel_found = self.net.selections[self.selection_key]['selection_found']
        if self.sel_found:
            self.selected_links = self.net.selections[self.selection_key]['selected_links']
            self.sp_route = self.net.selections[self.selection_key]['route']        
    
    def layout(self, map):
        '''
        grrr not working
        '''
        srcdoc = map._repr_html_().replace('"', '&quot;')
        html     = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: 400px;'
                 'border: none"></iframe>'.format(srcdoc))
        
        return VBox(display(self.links), display(html), display(self.query))
    
    def folium_node(self, node, node_name, color='white', icon = ''):
        node_marker = folium.Marker(location=[node['y'],node['x']], 
                                    icon=folium.Icon(icon = icon, color=color), 
                                    tooltip=node_name,
                                   )
        return node_marker
    
    def folium_route(self):
        '''
        shows the sp route along which the selection is being created
        '''
        m = ox.plot_route_folium(self.G, self.sp_route, route_map=self.folium_graph(),
                          route_color='white', route_width=15, route_opacity=.5)
        return m
    
    def folium_selection(self):
        '''
        Shows which links are selected
        '''
        m = self.folium_graph()
        
        for _, row in self.net.selections[self.selection_key]['selected_links'].iterrows():
            pl = ox.plot.make_folium_polyline(edge=row, edge_color="green", edge_width=7,
                                      edge_opacity=0.4)
            pl.add_to(m)
            
        
        return m
    
    def folium_graph(self):
        '''
        Shows the entirety of the search subgraph
        '''
        m = ox.plot_graph_folium(self.G, 
                                 edge_color='blue',
                                 edge_width=5, 
                                 edge_opacity = 0.1,
                                 tiles='cartodbdark_matter')
        self.folium_node(self.A, self.A_name, color="green", icon = 'play').add_to(m)
        self.folium_node(self.B, self.B_name, color="pink", icon = 'star').add_to(m)
        
        return m

    
    def candidate_graph(self):
        '''
        To review in case it doesn't find the SP
        ##todo doesn't work.
        '''
        m = self.folium_graph()
        
        links = ox.save_load.graph_to_gdfs(self.G, nodes=False, fill_edge_geometry=True)
        colors = ox.plot.get_colors(10)
        colors_d = {c:value for c, value in enumerate(colors, 0)}
        links['color'] = links['i'].map(colors_d)
        #print(links[['i','color']])
        
        for _, row in links.iterrows():
            pl = ox.plot.make_folium_polyline(edge=row, edge_color=row['color'], edge_width=5,
                                      edge_opacity=1.0)
            pl.add_to(m)
        self.folium_node(self.A, self.A_name, color="green").add_to(m)
        self.folium_node(self.B, self.B_name, color="pink").add_to(m)
        
        return m

In [81]:
sel = selection_map(net, selection=easy_selection)

graph_map = sel.folium_graph()
route_map = sel.folium_route()
#sel.layout(route_map)
#sel.candidate_graph()
selected_map = sel.folium_selection()
#route_map
#sel.links
selected_map
